In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input , Dense
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import Sequence
import bz2
import numpy as np
import pickle
import mmap 
import re

In [3]:

MAX_FEATURES = 12000
sequences = Input(shape=(255,))
embedded = layers.Embedding(MAX_FEATURES, 32)(sequences)
x = layers.Conv1D(64, 3, activation='relu')(embedded)
x = layers.BatchNormalization()(x)
x = layers.MaxPool1D(3)(x)
x = layers.Conv1D(32, 5, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(16, 5, activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

In [4]:
model = Model(inputs=sequences, outputs=outputs)

In [5]:
mch = callbacks.ModelCheckpoint('../models/fianl-model-cnn.h5' , monitor='accuracy' , mode ='max' , save_best_only=True)


In [6]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
with open('/content/drive/MyDrive/persian-sentiment-analysis/train_pad_sequences1.pickle' , 'rb') as f:
  seq_1 = pickle.load(f)

with open('/content/drive/MyDrive/persian-sentiment-analysis/train_pad_sequences2.pickle' , 'rb') as f:
  seq_2 = pickle.load(f)

with open('/content/train_pad_sequences.pickle' , 'wb') as f:
  pickle.dump(np.concatenate((seq_1 , seq_2), axis=0) , f)

del seq_1
del seq_2

In [8]:
class DataGenerator(Sequence):
    def __init__(self , dataset_size ,x_file, y_file, batch_size=32):
        self.dataset_size = dataset_size
        self.x_file = x_file
        self.y_file = y_file
        self.batch_size = batch_size


        with open(x_file, 'rb') as f:
          mm = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
          self.arr_train = np.load(mm , allow_pickle=True)
        

        with open(y_file, 'rb') as f:
          mm = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
          self.arr_y = np.load(mm , allow_pickle=True)


    def __len__(self):
        return int(np.ceil((self.dataset_size) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_X = self.arr_train[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.arr_y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return batch_X, batch_y
batch_size = 128
dataset_size_train = 3600000
dataset_size_test = 400000
train_generator = DataGenerator(dataset_size_train,"/content/train_pad_sequences.pickle", "/content/drive/MyDrive/persian-sentiment-analysis/train_labels.pickle", batch_size=batch_size)
test_generator = DataGenerator(dataset_size_test,"/content/drive/MyDrive/persian-sentiment-analysis/test_pad_sequences.pickle", "/content/drive/MyDrive/persian-sentiment-analysis/test_labels.pickle", batch_size=batch_size)

In [9]:
model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=10 , validation_data=test_generator, validation_steps=len(test_generator),callbacks=[mch])

Epoch 1/10
28125/28125 [==============================] - 510s 18ms/step - loss: 0.1685 - accuracy: 0.9348 - val_loss: 0.1463 - val_accuracy: 0.9456
Epoch 2/10
28125/28125 [==============================] - 288s 10ms/step - loss: 0.1361 - accuracy: 0.9496 - val_loss: 0.1389 - val_accuracy: 0.9484
Epoch 3/10
28125/28125 [==============================] - 289s 10ms/step - loss: 0.1257 - accuracy: 0.9540 - val_loss: 0.1372 - val_accuracy: 0.9491
Epoch 4/10
28125/28125 [==============================] - 287s 10ms/step - loss: 0.1181 - accuracy: 0.9572 - val_loss: 0.1403 - val_accuracy: 0.9480
Epoch 5/10
28125/28125 [==============================] - 287s 10ms/step - loss: 0.1121 - accuracy: 0.9599 - val_loss: 0.1416 - val_accuracy: 0.9488
Epoch 6/10
28125/28125 [==============================] - 293s 10ms/step - loss: 0.1071 - accuracy: 0.9620 - val_loss: 0.1433 - val_accuracy: 0.9477
Epoch 7/10
28125/28125 [==============================] - 288s 10ms/step - loss: 0.1029 - accuracy: 0.9638